In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import cv2
import os
import glob

import functions as fn

import tensorflow as tf
from tensorflow import keras

seed=42

In [2]:
train_data = keras.preprocessing.image_dataset_from_directory(
    '../created_data/train', 
    labels='inferred',
    subset="training",
    validation_split=.2,
    seed=seed,
    shuffle=False)

val_data = keras.preprocessing.image_dataset_from_directory(
    '../created_data/train', 
    labels='inferred',
    subset="validation",
    validation_split=.2,
    seed=seed,
    shuffle=False)

test_data = keras.preprocessing.image_dataset_from_directory(
    '../created_data/test', 
    labels='inferred',
    shuffle=False)

Found 317 files belonging to 2 classes.
Using 254 files for training.
Found 317 files belonging to 2 classes.
Using 63 files for validation.
Found 78 files belonging to 2 classes.


In [3]:

for images, labels in train_data.take(1):
    for i in range(3):
        fn.display_image(images[i].numpy().astype('uint8'))

In [4]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224,224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [5]:
train_data = val_data.map(preprocess).prefetch(1)
val_data = val_data.map(preprocess).prefetch(1)
test_data = test_data.map(preprocess).prefetch(1)

In [6]:
base_model = keras.applications.xception.Xception(weights = 'imagenet', include_top = False)

for layer in base_model.layers:
    layer.trainable = False

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(1, activation = 'sigmoid')(avg)
model = keras.Model(inputs = base_model.input, outputs = output)

optimizer = keras.optimizers.SGD(lr = 0.2, momentum = 0.9, decay = 0.01)
model.compile(loss = 'binary_crossentropy', optimizer = optimizer,  metrics = ['accuracy', tf.keras.metrics.Recall()])

results = model.fit(train_data, epochs = 5, validation_data = val_data)

Epoch 1/5


c:\Users\nicho\anaconda3\envs\capstone-env\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


2/2 [==============================] - 5s 2s/step - loss: 0.2993 - accuracy: 0.9206 - recall: 0.9206 - val_loss: 5.2499e-06 - val_accuracy: 1.0000 - val_recall: 1.0000
Epoch 2/5
2/2 [==============================] - 2s 2s/step - loss: 1.1529e-06 - accuracy: 1.0000 - recall: 1.0000 - val_loss: 2.5658e-09 - val_accuracy: 1.0000 - val_recall: 1.0000
Epoch 3/5
2/2 [==============================] - 2s 2s/step - loss: 2.5134e-10 - accuracy: 1.0000 - recall: 1.0000 - val_loss: 6.2590e-12 - val_accuracy: 1.0000 - val_recall: 1.0000
Epoch 4/5
2/2 [==============================] - 2s 2s/step - loss: 6.1702e-13 - accuracy: 1.0000 - recall: 1.0000 - val_loss: 4.8905e-14 - val_accuracy: 1.0000 - val_recall: 1.0000
Epoch 5/5
2/2 [==============================] - 2s 2s/step - loss: 6.6578e-15 - accuracy: 1.0000 - recall: 1.0000 - val_loss: 9.6554e-16 - val_accuracy: 1.0000 - val_recall: 1.0000


In [8]:
model.evaluate(test_data)

3/3 [==============================] - 2s 435ms/step - loss: 27.9444 - accuracy: 0.4872 - recall: 1.0000


[27.944395065307617, 0.4871794879436493, 1.0]